In [10]:
# Run a mock of fractal-demos/examples/01, with an additional illumination correction task.

In [11]:
# Preliminary imports and auxiliary-function definition

from pathlib import Path
from devtools import debug
import tempfile

from fractal_server.app.runner.v2.models import Dataset, WorkflowTask
from fractal_server.app.runner.v2.runner import execute_tasks_v2
from fractal_server.images import SingleImage

import sys
sys.path.append("../tests/v2/04_runner")
from fractal_tasks_core_mock import TASK_LIST

import sys
sys.path.append("../tests/v2/04_runner")
from fractal_tasks_core_mock import TASK_LIST

from concurrent.futures import ThreadPoolExecutor

executor = ThreadPoolExecutor()


def image_data_exist_on_disk(image_list: list[SingleImage]):
    """
    Given an image list, check whether mock data were written to disk.
    """
    all_images_have_data = True
    for image in image_list:
        if (Path(image.path) / "data").exists():
            print(f"[image_data_exist_on_disk] {image.path} contains data")
        else:
            print(f"[image_data_exist_on_disk] {image.path} does *not* contain data")
            all_images_have_data = False
        
    return all_images_have_data

In [12]:
# Create temporary directory for mocked zarrs
tmp_path = Path(tempfile.mkdtemp())
zarr_dir = (tmp_path / "zarr_dir").as_posix().rstrip("/")
print(f"{zarr_dir=}")

zarr_dir='/tmp/tmpbin3lktp/zarr_dir'


In [13]:
# Run create-ome-zarr (starting from empty Dataset)
dataset = execute_tasks_v2(
    wf_task_list=[
        WorkflowTask(
            task=TASK_LIST["create_ome_zarr_compound"],
            args_non_parallel=dict(image_dir="/tmp/input_images"),
        )
    ],
    dataset=Dataset(zarr_dir=zarr_dir),
    executor=executor,
)

# Images were added, but data do not exist on-disk yet
assert image_data_exist_on_disk(dataset.images)

# Print dataset metadata
debug(dataset);

[create_ome_zarr] START
[create_ome_zarr] image_dir='/tmp/input_images'
[create_ome_zarr] zarr_dir='/tmp/tmpbin3lktp/zarr_dir'
[create_ome_zarr] zarr_path='/tmp/tmpbin3lktp/zarr_dir/my_plate.zarr'
[create_ome_zarr] END
[yokogawa_to_zarr] START
[yokogawa_to_zarr] path='/tmp/tmpbin3lktp/zarr_dir/my_plate.zarr/A/01/0'
[yokogawa_to_zarr] raw_path='/tmp/input_images/A_01_0.tif'
[yokogawa_to_zarr] START
[yokogawa_to_zarr] path='/tmp/tmpbin3lktp/zarr_dir/my_plate.zarr/A/02/0'
[yokogawa_to_zarr] raw_path='/tmp/input_images/A_02_0.tif'
[yokogawa_to_zarr] END
[yokogawa_to_zarr] END
[image_data_exist_on_disk] /tmp/tmpbin3lktp/zarr_dir/my_plate.zarr/A/01/0 contains data
[image_data_exist_on_disk] /tmp/tmpbin3lktp/zarr_dir/my_plate.zarr/A/02/0 contains data
/tmp/ipykernel_125078/61380797.py:17 <module>
    dataset: Dataset(
        id=None,
        history=[None],
        zarr_dir='/tmp/tmpbin3lktp/zarr_dir',
        images=[
            SingleImage(
                path='/tmp/tmpbin3lktp/zarr_dir/

In [14]:
# Run illumination correction, with overwrite_input=True
dataset = execute_tasks_v2(
    wf_task_list=[
        WorkflowTask(
            task=TASK_LIST["illumination_correction"],
            args_parallel=dict(overwrite_input=True),
        )
    ],
    dataset=dataset,
    executor=executor,
)

# Image data still exist (and they actually were modified)
assert image_data_exist_on_disk(dataset.images)

# Print dataset metadata
debug(dataset);

[illumination_correction] START
[illumination_correction] path='/tmp/tmpbin3lktp/zarr_dir/my_plate.zarr/A/01/0'
[illumination_correction] overwrite_input=True
[illumination_correction] START
[illumination_correction] path='/tmp/tmpbin3lktp/zarr_dir/my_plate.zarr/A/02/0'
[illumination_correction] overwrite_input=True
[illumination_correction] END
[illumination_correction] END
[new_ome_zarr] START
[new_ome_zarr] paths=['/tmp/tmpbin3lktp/zarr_dir/my_plate.zarr/A/01/0', '/tmp/tmpbin3lktp/zarr_dir/my_plate.zarr/A/02/0']
[new_ome_zarr] Identified old_plate='my_plate.zarr'
[new_ome_zarr] new_plate='my_plate_mip.zarr'
[new_ome_zarr] new_zarr_path='/tmp/tmpbin3lktp/zarr_dir/my_plate_mip.zarr'
[new_ome_zarr] END
[maximum_intensity_projection] START
[maximum_intensity_projection] path='/tmp/tmpbin3lktp/zarr_dir/my_plate.zarr/A/02/0'
[maximum_intensity_projection] new_path='/tmp/tmpbin3lktp/zarr_dir/my_plate_mip.zarr/A/02/0'
[maximum_intensity_projection] END
[maximum_intensity_projection] START
[

[image_data_exist_on_disk] /tmp/tmpbin3lktp/zarr_dir/my_plate.zarr/A/01/0 contains data
[image_data_exist_on_disk] /tmp/tmpbin3lktp/zarr_dir/my_plate.zarr/A/02/0 contains data
/tmp/ipykernel_125078/4204713964.py:17 <module>
    dataset: Dataset(
        id=None,
        history=[
            None,
            'illumination_correction',
        ],
        zarr_dir='/tmp/tmpbin3lktp/zarr_dir',
        images=[
            SingleImage(
                path='/tmp/tmpbin3lktp/zarr_dir/my_plate.zarr/A/01/0',
                attributes={
                    'well': 'A01',
                    'plate': 'my_plate.zarr',
                    'data_dimensionality': 3,
                    'illumination_correction': True,
                },
            ),
            SingleImage(
                path='/tmp/tmpbin3lktp/zarr_dir/my_plate.zarr/A/02/0',
                attributes={
                    'well': 'A02',
                    'plate': 'my_plate.zarr',
                    'data_dimensionality': 

In [15]:
# Run new-ome-zarr, as an init task for MIP
dataset = execute_tasks_v2(
    wf_task_list=[
        WorkflowTask(
            task=TASK_LIST["MIP_compound"],
            args_non_parallel=dict(suffix="mip"),
        )
    ],
    dataset=dataset,
    executor=executor,
)

# 3D image data exist on disk
images_3D = [
        image
        for image in dataset.images
        if image.attributes["data_dimensionality"] == 3
    ]
assert image_data_exist_on_disk(images_3D)

# 2D image data do not exist on disk
images_2D = [
        image
        for image in dataset.images
        if image.attributes["data_dimensionality"] == 2
    ]
assert image_data_exist_on_disk(images_2D)

# Print dataset metadata
debug(dataset);

[image_data_exist_on_disk] /tmp/tmpbin3lktp/zarr_dir/my_plate.zarr/A/01/0 contains data
[image_data_exist_on_disk] /tmp/tmpbin3lktp/zarr_dir/my_plate.zarr/A/02/0 contains data
[image_data_exist_on_disk] /tmp/tmpbin3lktp/zarr_dir/my_plate_mip.zarr/A/01/0 contains data
[image_data_exist_on_disk] /tmp/tmpbin3lktp/zarr_dir/my_plate_mip.zarr/A/02/0 contains data
/tmp/ipykernel_125078/2260398909.py:30 <module>
    dataset: Dataset(
        id=None,
        history=[
            None,
            'illumination_correction',
            None,
        ],
        zarr_dir='/tmp/tmpbin3lktp/zarr_dir',
        images=[
            SingleImage(
                path='/tmp/tmpbin3lktp/zarr_dir/my_plate.zarr/A/01/0',
                attributes={
                    'well': 'A01',
                    'plate': 'my_plate.zarr',
                    'data_dimensionality': 3,
                    'illumination_correction': True,
                },
            ),
            SingleImage(
                path='

In [16]:
# Run cellpose on current images
dataset = execute_tasks_v2(
    wf_task_list=[
        WorkflowTask(
            task=TASK_LIST["cellpose_segmentation"],
            args_parallel=dict(),
        )
    ],
    dataset=dataset,
    executor=executor,
)

# All image data (3D and 2D) now exists on disk
assert image_data_exist_on_disk(dataset.images)

# Print dataset metadata
debug(dataset);

[image_data_exist_on_disk] /tmp/tmpbin3lktp/zarr_dir/my_plate.zarr/A/01/0 contains data
[image_data_exist_on_disk] /tmp/tmpbin3lktp/zarr_dir/my_plate.zarr/A/02/0 contains data
[image_data_exist_on_disk] /tmp/tmpbin3lktp/zarr_dir/my_plate_mip.zarr/A/01/0 contains data
[image_data_exist_on_disk] /tmp/tmpbin3lktp/zarr_dir/my_plate_mip.zarr/A/02/0 contains data
/tmp/ipykernel_125078/1519895496.py:17 <module>
    dataset: Dataset(
        id=None,
        history=[
            None,
            'illumination_correction',
            None,
            'cellpose_segmentation',
        ],
        zarr_dir='/tmp/tmpbin3lktp/zarr_dir',
        images=[
            SingleImage(
                path='/tmp/tmpbin3lktp/zarr_dir/my_plate.zarr/A/01/0',
                attributes={
                    'well': 'A01',
                    'plate': 'my_plate.zarr',
                    'data_dimensionality': 3,
                    'illumination_correction': True,
                },
            ),
          

In [10]:
executor.shutdown()